Considere uma cadeia circular (primeira massa conectada a última) composta por N átomos conectados por molas idênticas de constante elástica k . O sistema pode ser de dois tipos:
• Cadeia Ternária: As massas alternam entre m e m2=3m e m3=5m  ao longo da cadeia.
• Cadeia Homogênea: Todas as massas são iguais a m .
Ambos os sistemas devem ser analisados sob condições de contorno com pontas livres (as extremidades da cadeia não estão presas).

In [2]:
#Biblioteca
import numpy as np
import matplotlib.pyplot as plt

Parâmetros
• Massas: m= 1kg , 
• Constante elástica:  k = 1N/m
• Tamanhos da cadeia: N = 100, 1000 e 10.000

In [3]:
m = 1
k = 1
N1 = 100
N2 = 1000
N3 = 10000

1. Cálculo das Frequências de Vibração
• Para cada valor de N , calcule as frequências naturais de vibração para:
• Cadeia ternária(alternância  --m--m2--m3--m--m2--m3-- )
• Cadeia homogênea (todas as massas iguais)
• Considere sempre as pontas livres.

In [ ]:
## cadeia ternaria


## cadeia homegenea

2. Densidade de Estados
• Construa histogramas (densidade de estados) para os valores das frequências em cada caso.
• Compare os histogramas entre a cadeia binária e a homogênea, para cada tamanho de N .

In [ ]:
## geração dos histogramas de densidade de estados

3. Deslocamentos Relativos (Modos Normais)
• Para cada cadeia, obtenha os vetores de deslocamento (modos normais) correspondentes:
• Às cinco menores frequências (modos de baixa energia)
• Às cinco maiores frequências (modos de alta energia)
• Apresente gráficos dos deslocamentos relativos dos átomos para esses modos.

In [4]:
## adaptar função para o problema
def resolver_autovalor(M, K):
    #Calcular a serie harmonica e seu inverso
    Minv = np.linalg.inv(M)
    A = Minv @ K
    # Calculando os autovalores e autovetores
    eigvals, eigvecs = np.linalg.eig(A)
    # Frequências naturais
    omegas = np.sqrt(np.clip(eigvals.real, 0, None))
    ##ordenação de forma crescente para facilitar a visualização (na saida)
    idx = np.argsort(omegas)
    return omegas[idx], eigvecs[:, idx]

4. Análise e Comparação
• Analise como a alternância de massas (ternária) e a homogeneidade afetam:
• A distribuição das frequências (densidade de estados)
• Os padrões de deslocamento dos modos normais
• Discuta a presença de possíveis lacunas de frequência (band gaps) e a localização dos modos.